In [1]:
import os
import datetime
import numpy as np
import cv2
import random
from PIL import Image
from functools import partial
from tqdm import tqdm
import math

import torch
from torch import nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from nets.deeplabv3_plus import DeepLab

## partition dataset

In [2]:
# 划分比例
train_percent       = 0.85
dataset_path      = 'weizmann_horse_db'
random.seed(0)

segfilepath     = os.path.join(dataset_path, 'mask')
saveBasePath    = os.path.join(dataset_path, 'datasets')

In [3]:
temp_seg = os.listdir(segfilepath)
total_seg = []
for seg in temp_seg:
    if seg.endswith(".png"):
        total_seg.append(seg)

num = len(total_seg)  
list = range(num)  
tv = int(num)
tr = int(tv*train_percent)  
trainval = random.sample(list,tv)  
train = random.sample(trainval,tr)  

In [4]:
#划分结果存入 weizmann_horse_db/datasets
ftrainval = open(os.path.join(saveBasePath,'trainval.txt'), 'w')  
ftest = open(os.path.join(saveBasePath,'test.txt'), 'w')  
ftrain = open(os.path.join(saveBasePath,'train.txt'), 'w')  
fval = open(os.path.join(saveBasePath,'val.txt'), 'w')  

for i in list:  
    name = total_seg[i][:-4]+'\n'  
    if i in trainval:  
        ftrainval.write(name)  
        if i in train:  
            ftrain.write(name)  
        else:  
            fval.write(name)  
    else:  
        ftest.write(name)  

ftrainval.close()  
ftrain.close()  
fval.close()  
ftest.close()

print("train size",tr)

train size 277


## load model and set logs

In [5]:
Cuda = True
num_classes = 2
model_path = "model/deeplabv3+_model.pth"
dataset_path = 'weizmann_horse_db'
save_dir = 'logs'

input_shape = [512, 512]

Epoch = 1
batch_size = 4

In [6]:
# 初始话模型权重
def weights_init(net):
    def init_func(m):
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and classname.find('Conv') != -1:
            torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm2d') != -1:
            torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
            torch.nn.init.constant_(m.bias.data, 0.0)
    net.apply(init_func)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DeepLab(num_classes=num_classes)
weights_init(model)

# 根据预训练权重的Key和模型的Key进行加载
model_dict      = model.state_dict()
pretrained_dict = torch.load(model_path, map_location = device)

load_key, no_load_key, temp_dict = [], [], {}
for k, v in pretrained_dict.items():
    if k in model_dict.keys() and np.shape(model_dict[k]) == np.shape(v):
        temp_dict[k] = v
        load_key.append(k)
    else:
        no_load_key.append(k)
model_dict.update(temp_dict)
model.load_state_dict(model_dict)
print('The numbers of keys which fail loading:',len(no_load_key))

The numbers of keys which fail loading: 0


In [7]:
class LossHistory():
    def __init__(self, log_dir, model, input_shape):
        self.log_dir = log_dir
        self.losses = []
        self.val_loss = []

        os.makedirs(self.log_dir)
        self.writer = SummaryWriter(self.log_dir)
        try:
            dummy_input = torch.randn(2, 3, input_shape[0], input_shape[1])
            self.writer.add_graph(model, dummy_input)
        except:
            pass

    def append_loss(self, epoch, loss, val_loss):
        if not os.path.exists(self.log_dir):
            os.makedirs(self.log_dir)

        self.losses.append(loss)
        self.val_loss.append(val_loss)

        self.writer.add_scalar('loss', loss, epoch)
        self.writer.add_scalar('val_loss', val_loss, epoch)

In [8]:
# 日志保存
time_str = datetime.datetime.strftime(datetime.datetime.now(),'%Y_%m_%d_%H_%M_%S')
log_dir = os.path.join(save_dir, "loss_" + str(time_str))
loss_history = LossHistory(log_dir, model, input_shape=input_shape)
model_train = model.train()

In [9]:
if Cuda:
    model_train = torch.nn.DataParallel(model)
    cudnn.benchmark = True
    model_train = model_train.cuda()

## load dataset

In [10]:
def preprocess_input(image):
    image /= 255.0
    return image

def cvtColor(image):
    if len(np.shape(image)) == 3 and np.shape(image)[2] == 3:
        return image
    else:
        image = image.convert('RGB')
        return image

In [11]:
# 载入数据集，对训练图数据增强
class DeeplabDataset():
    def __init__(self, annotation_lines, input_shape, num_classes, train, dataset_path):
        super(DeeplabDataset, self).__init__()
        self.annotation_lines = annotation_lines
        self.length = len(annotation_lines)
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.train = train
        self.dataset_path = dataset_path

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        annotation_line = self.annotation_lines[index]
        name = annotation_line.split()[0]

        # 从文件中读取图像
        jpg = Image.open(os.path.join(os.path.join(self.dataset_path, "horse"), name + ".jpg"))
        png = Image.open(os.path.join(os.path.join(self.dataset_path, "mask"), name + ".png"))
        # 数据增强
        jpg, png = self.get_random_data(jpg, png, self.input_shape, random=self.train)

        jpg = np.transpose(preprocess_input(np.array(jpg, np.float64)), [2, 0, 1])
        png = np.array(png)
        png[png >= self.num_classes] = self.num_classes
        # 转化成one_hot的形式 ,在这里需要+1是因为数据集有些标签具有白边部分, 我们需要将白边部分进行忽略，+1的目的是方便忽略。
        seg_labels = np.eye(self.num_classes + 1)[png.reshape([-1])]
        seg_labels = seg_labels.reshape((int(self.input_shape[0]), int(self.input_shape[1]), self.num_classes + 1))

        return jpg, png, seg_labels

    def rand(self, a=0, b=1):
        return np.random.rand() * (b - a) + a

    def get_random_data(self, image, label, input_shape, jitter=.3, hue=.1, sat=0.7, val=0.3, random=True):
        image = cvtColor(image)
        label = Image.fromarray(np.array(label))
        # 获得图像的高宽与目标高宽
        iw, ih = image.size
        h, w = input_shape

        if not random:
            iw, ih = image.size
            scale = min(w / iw, h / ih)
            nw = int(iw * scale)
            nh = int(ih * scale)

            image = image.resize((nw, nh), Image.BICUBIC)
            new_image = Image.new('RGB', [w, h], (128, 128, 128))
            new_image.paste(image, ((w - nw) // 2, (h - nh) // 2))

            label = label.resize((nw, nh), Image.NEAREST)
            new_label = Image.new('L', [w, h], (0))
            new_label.paste(label, ((w - nw) // 2, (h - nh) // 2))
            return new_image, new_label

        # 对图像进行缩放并且进行长和宽的扭曲
        new_ar = iw / ih * self.rand(1 - jitter, 1 + jitter) / self.rand(1 - jitter, 1 + jitter)
        scale = self.rand(0.25, 2)
        if new_ar < 1:
            nh = int(scale * h)
            nw = int(nh * new_ar)
        else:
            nw = int(scale * w)
            nh = int(nw / new_ar)
        image = image.resize((nw, nh), Image.BICUBIC)
        label = label.resize((nw, nh), Image.NEAREST)

        # 翻转图像
        flip = self.rand() < .5
        if flip:
            image = image.transpose(Image.FLIP_LEFT_RIGHT)
            label = label.transpose(Image.FLIP_LEFT_RIGHT)

        # 将图像多余的部分加上灰条
        dx = int(self.rand(0, w - nw))
        dy = int(self.rand(0, h - nh))
        new_image = Image.new('RGB', (w, h), (128, 128, 128))
        new_label = Image.new('L', (w, h), (0))
        new_image.paste(image, (dx, dy))
        new_label.paste(label, (dx, dy))
        image = new_image
        label = new_label

        image_data = np.array(image, np.uint8)

        # 高斯模糊
        blur = self.rand() < 0.25
        if blur:
            image_data = cv2.GaussianBlur(image_data, (5, 5), 0)

        # 旋转
        rotate = self.rand() < 0.25
        if rotate:
            center = (w // 2, h // 2)
            rotation = np.random.randint(-10, 11)
            M = cv2.getRotationMatrix2D(center, -rotation, scale=1)
            image_data = cv2.warpAffine(image_data, M, (w, h), flags=cv2.INTER_CUBIC, borderValue=(128, 128, 128))
            label = cv2.warpAffine(np.array(label, np.uint8), M, (w, h), flags=cv2.INTER_NEAREST, borderValue=(0))

        # 对图像进行色域变换 ,计算色域变换的参数
        r = np.random.uniform(-1, 1, 3) * [hue, sat, val] + 1
        # 将图像转到HSV上
        hue, sat, val = cv2.split(cv2.cvtColor(image_data, cv2.COLOR_RGB2HSV))
        dtype = image_data.dtype
        # 应用变换
        x = np.arange(0, 256, dtype=r.dtype)
        lut_hue = ((x * r[0]) % 180).astype(dtype)
        lut_sat = np.clip(x * r[1], 0, 255).astype(dtype)
        lut_val = np.clip(x * r[2], 0, 255).astype(dtype)

        image_data = cv2.merge((cv2.LUT(hue, lut_hue), cv2.LUT(sat, lut_sat), cv2.LUT(val, lut_val)))
        image_data = cv2.cvtColor(image_data, cv2.COLOR_HSV2RGB)

        return image_data, label

In [12]:
def collate_function(batch):
    images = []
    pngs = []
    seg_labels = []
    for img, png, labels in batch:
        images.append(img)
        pngs.append(png)
        seg_labels.append(labels)
    images = torch.from_numpy(np.array(images)).type(torch.FloatTensor)
    pngs = torch.from_numpy(np.array(pngs)).long()
    seg_labels = torch.from_numpy(np.array(seg_labels)).type(torch.FloatTensor)
    return images, pngs, seg_labels

In [13]:
with open(os.path.join(dataset_path, "datasets/train.txt"),"r") as f:
        train_lines = f.readlines()
with open(os.path.join(dataset_path, "datasets/val.txt"),"r") as f:
        val_lines = f.readlines()
num_train = len(train_lines)
num_val   = len(val_lines)

train_dataset = DeeplabDataset(train_lines, input_shape, num_classes, True, dataset_path)
val_dataset = DeeplabDataset(val_lines, input_shape, num_classes, False, dataset_path)

train_set = DataLoader(train_dataset, shuffle = True, batch_size = batch_size, pin_memory=True,
                 drop_last = True, collate_fn = collate_function)
val_set = DataLoader(val_dataset  , shuffle = True, batch_size = batch_size, pin_memory=True,
                     drop_last = True, collate_fn = collate_function)
# 判断每一个epoch的长度
epoch_step = num_train // batch_size
epoch_step_val = num_val // batch_size

## optimizer and learning rate

In [14]:
def get_lr_scheduler(lr, min_lr, total_iters):
    warmup_iters_ratio = 0.1
    warmup_lr_ratio = 0.1
    no_aug_iter_ratio = 0.3
    def warm_cos_lr(lr, min_lr, total_iters, warmup_total_iters, warmup_lr_start, no_aug_iter, iters):
        if iters <= warmup_total_iters:
            lr = (lr - warmup_lr_start) * pow(iters / float(warmup_total_iters), 2) + warmup_lr_start
        elif iters >= total_iters - no_aug_iter:
            lr = min_lr
        else:
            lr = min_lr + 0.5 * (lr - min_lr) * (
                1.0 + math.cos(math.pi* (iters - warmup_total_iters) / (total_iters - warmup_total_iters - no_aug_iter))
            )
        return lr

    warmup_total_iters = min(max(warmup_iters_ratio * total_iters, 1), 3)
    warmup_lr_start = max(warmup_lr_ratio * lr, 1e-6)
    no_aug_iter = min(max(no_aug_iter_ratio * total_iters, 1), 15)
    func = partial(warm_cos_lr, lr, min_lr, total_iters, warmup_total_iters, warmup_lr_start, no_aug_iter)
    return func

def set_optimizer_lr(optimizer, lr_scheduler_func, epoch):
    lr = lr_scheduler_func(epoch)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [15]:
momentum = 0.9
weight_decay = 1e-4

Init_lr = 7e-3
Min_lr = Init_lr * 0.01

nbs  = 16
lr_limit_max = 1e-1
lr_limit_min = 5e-4
# 根据batch_size，自适应调整学习率
Init_lr_fit = min(max(batch_size / nbs * Init_lr, lr_limit_min), lr_limit_max)
Min_lr_fit = min(max(batch_size / nbs * Min_lr, lr_limit_min * 1e-2), lr_limit_max * 1e-2)
# SGD优化器
optimizer = optim.SGD(model.parameters(), Init_lr_fit, momentum = momentum, nesterov=True, weight_decay = weight_decay)
# 获得学习率下降的公式
lr_scheduler_func = get_lr_scheduler(Init_lr_fit, Min_lr_fit, Epoch)

## train

In [16]:
# 计算 交叉熵损失
def CE_Loss(inputs, target, cls_weights, num_classes=2):
    n, c, h, w = inputs.size()
    nt, ht, wt = target.size()
    if h != ht and w != wt:
        inputs = F.interpolate(inputs, size=(ht, wt), mode="bilinear", align_corners=True)

    temp_inputs = inputs.transpose(1, 2).transpose(2, 3).contiguous().view(-1, c)
    temp_target = target.view(-1)

    CE_loss  = nn.CrossEntropyLoss(weight=cls_weights, ignore_index=num_classes)(temp_inputs, temp_target)
    return CE_loss

# 计算Dice损失
def Dice_loss(inputs, target, beta=1, smooth = 1e-5):
    n, c, h, w = inputs.size()
    nt, ht, wt, ct = target.size()
    if h != ht and w != wt:
        inputs = F.interpolate(inputs, size=(ht, wt), mode="bilinear", align_corners=True)
        
    temp_inputs = torch.softmax(inputs.transpose(1, 2).transpose(2, 3).contiguous().view(n, -1, c),-1)
    temp_target = target.view(n, -1, ct)

    #   计算dice loss
    tp = torch.sum(temp_target[...,:-1] * temp_inputs, axis=[0,1])
    fp = torch.sum(temp_inputs , axis=[0,1]) - tp
    fn = torch.sum(temp_target[...,:-1] , axis=[0,1]) - tp

    score = ((1 + beta ** 2) * tp + smooth) / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + smooth)
    dice_loss = 1 - torch.mean(score)
    return dice_loss

In [17]:
def fit_one_epoch(model_train, model, loss_history, optimizer, epoch, Epoch, epoch_step, epoch_step_val,
                  train_set, val_set, cuda , num_classes, save_dir):
    train_loss = 0
    val_loss = 0

    cls_weights = np.ones([num_classes], np.float32)
    pbar = tqdm(total=epoch_step,desc=f'Epoch(train) {epoch + 1}/{Epoch}',postfix=dict)
    model_train.train()
    for iteration, batch in enumerate(train_set):
        if iteration >= epoch_step:
            break
        imgs, pngs, labels = batch

        with torch.no_grad():
            weights = torch.from_numpy(cls_weights)
            if cuda:
                imgs = imgs.cuda()
                pngs = pngs.cuda()
                labels = labels.cuda()
                weights = weights.cuda()
        #   清零梯度
        optimizer.zero_grad()
        #   前向传播
        outputs = model_train(imgs)
        #   计算损失
        loss = CE_Loss(outputs, pngs, weights, num_classes = num_classes)
        loss_dice = Dice_loss(outputs, labels)
        loss = loss + loss_dice

        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        pbar.set_postfix(**{'train_loss': train_loss / (iteration + 1)})
        pbar.update(1)
    pbar.close()

    pbar = tqdm(total=epoch_step_val, desc=f'Epoch(valid) {epoch + 1}/{Epoch}',postfix=dict)
    model_train.eval()
    for iteration, batch in enumerate(val_set):
        if iteration >= epoch_step_val:
            break
        imgs, pngs, labels = batch
        with torch.no_grad():
            weights = torch.from_numpy(cls_weights)
            if cuda:
                imgs = imgs.cuda()
                pngs = pngs.cuda()
                labels = labels.cuda()
                weights = weights.cuda()
            #   前向传播
            outputs = model_train(imgs)
            #   计算损失
            loss = CE_Loss(outputs, pngs, weights, num_classes = num_classes)
            loss_dice = Dice_loss(outputs, labels)
            loss  = loss + loss_dice

            val_loss += loss.item()

            pbar.set_postfix(**{'val_loss': val_loss / (iteration + 1)})
            pbar.update(1)
    pbar.close()

    loss_history.append_loss(epoch + 1, train_loss / epoch_step, val_loss / epoch_step_val)
    #   保存权值
    torch.save(model.state_dict(), os.path.join(save_dir, "best_model.pth"))

In [18]:
for epoch in range(Epoch):
    set_optimizer_lr(optimizer, lr_scheduler_func, epoch)
    fit_one_epoch(model_train, model, loss_history, optimizer, epoch, Epoch, epoch_step, epoch_step_val,
                  train_set, val_set, Cuda, num_classes, save_dir)
loss_history.writer.close()

Epoch(valid) 1/1: 100%|███████████████████████████████████████████████| 12/12 [00:02<00:00,  4.84it/s, val_loss=0.0974]


## generate validation mask

In [19]:
# 对输入图像进行resize
def resize_image(image, size):
    iw, ih = image.size
    w, h = size

    scale = min(w/iw, h/ih)
    nw = int(iw*scale)
    nh = int(ih*scale)

    image = image.resize((nw,nh), Image.BICUBIC)
    new_image = Image.new('RGB', size, (128,128,128))
    new_image.paste(image, ((w-nw)//2, (h-nh)//2))

    return new_image, nw, nh

In [20]:
# 生成validation mask
class Detection(object):
    _defaults = {"model_path" : 'logs/best_model.pth',"num_classes" : 2,
                 "input_shape" : [512, 512],"cuda" : True,
    }
    def __init__(self, **kwargs):
        self.__dict__.update(self._defaults)
        for name, value in kwargs.items():
            setattr(self, name, value)
        #   画框的颜色
        self.colors = [ (0, 0, 0), (128, 0, 0)]
        self.generate()
    #   获得所有的分类
    def generate(self):
        #   载入模型与权值
        self.net = DeepLab(num_classes=self.num_classes)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.net.load_state_dict(torch.load(self.model_path, map_location=device))
        self.net = self.net.eval()

        if self.cuda:
            self.net = nn.DataParallel(self.net)
            self.net = self.net.cuda()

    def get_detection(self, image):
        #   代码仅仅支持RGB图像的预测，所有其它类型的图像都会转化成RGB
        image = cvtColor(image)
        orininal_h = np.array(image).shape[0]
        orininal_w = np.array(image).shape[1]

        image_data, nw, nh = resize_image(image, (self.input_shape[1],self.input_shape[0]))
        #   添加上batch_size维度
        image_data = np.expand_dims(np.transpose(preprocess_input(np.array(image_data, np.float32)), (2, 0, 1)), 0)

        with torch.no_grad():
            images = torch.from_numpy(image_data)
            if self.cuda:
                images = images.cuda()

            pr = self.net(images)[0]
            #   取出每一个像素点的种类
            pr = F.softmax(pr.permute(1,2,0),dim = -1).cpu().numpy()
            #   将灰条部分截取掉
            pr = pr[int((self.input_shape[0] - nh) // 2) : int((self.input_shape[0] - nh) // 2 + nh), \
                    int((self.input_shape[1] - nw) // 2) : int((self.input_shape[1] - nw) // 2 + nw)]
            #   进行图片的resize
            pr = cv2.resize(pr, (orininal_w, orininal_h), interpolation = cv2.INTER_LINEAR)
            #   取出每一个像素点的种类
            pr = pr.argmax(axis=-1)

        image = Image.fromarray(np.uint8(pr))
        return image

## from hist to miou

In [21]:
# 设标签宽W，长H
def fast_hist(a, b, n):
    # a是转化成一维数组的标签，形状(H×W,)；b是转化成一维数组的预测结果，形状(H×W,)
    k = (a >= 0) & (a < n)
    # np.bincount计算了从0到n**2-1这n**2个数中每个数出现的次数，返回值形状(n, n)
    # 返回中，写对角线上的为分类正确的像素点
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n ** 2).reshape(n, n)

def per_class_iu(hist):
    return np.diag(hist) / np.maximum((hist.sum(1) + hist.sum(0) - np.diag(hist)), 1)

def cal_miou(gt_dir, pred_dir, png_name_list, num_classes):
    # 创建一个全是0的矩阵，是一个混淆矩阵
    hist = np.zeros((num_classes, num_classes))
    # 获得验证集标签路径列表，获得验证集图像分割结果路径列表，方便直接读取
    gt_imgs = [os.path.join(gt_dir, x + ".png") for x in png_name_list]
    pred_imgs = [os.path.join(pred_dir, x + ".png") for x in png_name_list]
    # 读取每一个（图片-标签）对
    for ind in range(len(gt_imgs)):
        # 读取一张图像分割结果，转化成numpy数组
        pred = np.array(Image.open(pred_imgs[ind]))
        # 读取一张对应的标签，转化成numpy数组
        label = np.array(Image.open(gt_imgs[ind]))
        # 如果图像分割结果与标签的大小不一样，这张图片就不计算
        if len(label.flatten()) != len(pred.flatten()):  
            print(
                'Skipping: len(gt) = {:d}, len(pred) = {:d}, {:s}, {:s}'.format(
                    len(label.flatten()), len(pred.flatten()), gt_imgs[ind],
                    pred_imgs[ind]))
            continue
        # 对一张图片计算2×2的hist矩阵，并累加
        hist += fast_hist(label.flatten(), pred.flatten(), num_classes)
    # 计算所有验证集图片的逐类别mIoU值
    miou = per_class_iu(hist)
    # 在所有验证集图像上求所有类别平均的mIoU值，计算时忽略NaN值
    return miou

## from mask to boundary iou

In [22]:
#mask--->boundary
def mask_to_boundary(mask, dilation_ratio=0.02):
    h, w = mask.shape
    img_diag = np.sqrt(h ** 2 + w ** 2) # 计算图像对角线长度
    dilation = int(round(dilation_ratio * img_diag))
    if dilation < 1:
        dilation = 1
    # Pad image so mask truncated by the image border is also considered as boundary.
    new_mask = cv2.copyMakeBorder(mask, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0)
    kernel = np.ones((3, 3), dtype=np.uint8)
    new_mask_erode = cv2.erode(new_mask, kernel, iterations=dilation)   
    # 因为之前向四周填充了0, 故而这里不再需要四周
    mask_erode = new_mask_erode[1 : h + 1, 1 : w + 1]   
    # G_d intersects G in the paper.
    return mask - mask_erode

#计算boundary iou
def boundary_iou(gt, dt, dilation_ratio=0.02):
    gt_boundary = mask_to_boundary(gt, dilation_ratio)
    dt_boundary = mask_to_boundary(dt, dilation_ratio)
    intersection = ((gt_boundary * dt_boundary) > 0).sum()
    union = ((gt_boundary + dt_boundary) > 0).sum()
    if union < 1:
    	return 0
    boundary_iou = intersection / union
    return boundary_iou

## calculate m_iou, boundary iou

In [24]:
num_classes = 2
#   指向数据集所在的文件夹
dataset_path = 'weizmann_horse_db'

image_ids = open(os.path.join(dataset_path, "datasets/val.txt"),'r').read().splitlines()
gt_dir = os.path.join(dataset_path, "mask/")
pred_dir = "detection"

boundary_iou_list = []
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

Detect = Detection()

for image_id in image_ids:
    image_path = os.path.join(dataset_path, "horse/"+image_id+".jpg")
    image = Image.open(image_path)
    image = Detect.get_detection(image)
    image.save(os.path.join(pred_dir, image_id + ".png"))
    
    gt_path = os.path.join(dataset_path, "mask/"+image_id+".png")
    img_gt = Image.open(gt_path)
    gt = np.array(img_gt)
    
    gt_path = os.path.join(pred_dir, image_id+".png")
    img_dt = Image.open(gt_path)
    dt = np.array(img_dt)
    
    b_iou = boundary_iou(gt, dt, dilation_ratio=0.02)
    boundary_iou_list.append(b_iou)
miou = cal_miou(gt_dir, pred_dir, image_ids, num_classes)
print("----------------------------")
print('===> MIoU:         ' + str(round(np.nanmean(miou) * 100, 2)))
print('===> boundary iou: ' + str(round(np.nanmean(boundary_iou_list) * 100, 2)))
print("----------------------------")

----------------------------
===> MIoU:         93.36
===> boundary iou: 71.49
----------------------------
